In [1]:
from parsing_on_batch import process_html_and_extract_data
from extract_table import create_dataframes
import json
import pandas as pd
import numpy as np 


In [2]:
header_df,body_df=create_dataframes()
llm_output_list= process_html_and_extract_data()

In [3]:
header_df[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,,Head,,,,,,,
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.


In [4]:
body_df[3].head(10)

,0,1,2,3,4,5,6,7,8,9,10,11
0,2-56,,,,,,,,,,,
1,Black-Oxide Alloy Steel,,,,,,,,,,,
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51
5,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12
6,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93
7,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08
8,"5/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91864A005,9.24
9,"3/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A079,9.22


In [5]:
print(llm_output_list[3])

{
  "thread_size": [
    "2-56"
  ],
  "material_surface": [
    "Black-Oxide Alloy Steel",
    "Zinc-Plated Alloy Steel"
  ]
}


In [ ]:
head_row_len=header_df[3].shape[0]

2


In [7]:
header_df[3].head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,,,,,Head,,,,,,,
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.


In [8]:
no_cols=header_df[3].shape[1]

In [9]:
parsed_dict = json.loads(llm_output_list[3])
keys = list(parsed_dict.keys())


In [ ]:
head_df=header_df[3]
for i in range(len(keys)):
    new_column_name = f"{no_cols + 1}" 
    head_df[new_column_name] = np.nan  
    head_df.loc[head_row_len-1, new_column_name] = keys[i]  
    no_cols += 1 

header_df[0]=head_df
    

/tmp/ipykernel_81152/831104954.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'thread_size' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  head_df.loc[1, new_column_name] = keys[i]
/tmp/ipykernel_81152/831104954.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'material_surface' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  head_df.loc[1, new_column_name] = keys[i]


In [11]:
header_df[3]

,0,1,2,3,4,5,6,7,8,9,10,11,13,14
0,,,,,Head,,,,,,,,NaN,NaN
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface


In [12]:
body_df[3].head()


,0,1,2,3,4,5,6,7,8,9,10,11
0,2-56,,,,,,,,,,,
1,Black-Oxide Alloy Steel,,,,,,,,,,,
2,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91
3,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00
4,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51


In [13]:
merged_df = pd.concat([header_df[3], body_df[3]], ignore_index=True)

In [14]:
merged_df.head(30)

,0,1,2,3,4,5,6,7,8,9,10,11,13,14
0,,,,,Head,,,,,,,,NaN,NaN
1,Lg.,Threading,Min.Thread Lg.,Thread Spacing,Dia.,Ht.,DriveSize,"TensileStrength, psi",Specifications Met,Pkg.Qty.,,Pkg.,thread_size,material_surface
2,2-56,,,,,,,,,,,,NaN,NaN
3,Black-Oxide Alloy Steel,,,,,,,,,,,,NaN,NaN
4,"1/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A213,$17.91,NaN,NaN
5,"3/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000","ASME B18.3, ASTM A574",5,91251A470,19.00,NaN,NaN
6,"1/8""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,25,91251A072,11.51,NaN,NaN
7,"5/32""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,10,91864A010,11.12,NaN,NaN
8,"3/16""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A076,8.93,NaN,NaN
9,"1/4""",Fully Threaded,__,Coarse,"9/64""","0.086""","5/64""","170,000",ASTM A574,100,91251A077,9.08,NaN,NaN
